In [11]:
from rdflib import Namespace, Graph, URIRef
from rdflib.namespace import NamespaceManager, OWL, RDF, RDFS, XSD

In [12]:
# Get last element which is the base uri
def getNamespace(g):
    namespaces = g.namespaces()
    base_prefix = ''
    base_uri = ''

    for prefix, uri in namespaces:
        base_prefix, base_uri = prefix, uri
    return (base_prefix, base_uri)


In [1]:
def connectIndividuals(g, prefix, uri, news_name, news_class):
    query = f"""
    PREFIX {prefix}: <{uri}>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    INSERT {{
        {prefix}:{news_class} a rdfs:Class ;
            rdfs:label "{news_class}" .

        {prefix}:relatedTo a owl:ObjectProperty ;
            rdfs:label "related to" ;
            rdfs:domain {prefix}:{news_class} .

        {prefix}:{news_name} a {prefix}:{news_class} ;
            {prefix}:relatedTo ?otherIndiv .
    }}
    WHERE {{
        SELECT ?otherIndiv
        WHERE {{
            ?otherIndiv a ?type .
            FILTER(?otherIndiv != {prefix}:{news_name})
            FILTER NOT EXISTS {{ ?otherIndiv rdf:type rdfs:Class }}
            FILTER NOT EXISTS {{ ?otherIndiv rdf:type rdf:Property }}
            FILTER NOT EXISTS {{ ?otherIndiv rdf:type owl:Ontology }}
            FILTER NOT EXISTS {{ ?otherIndiv rdf:type owl:ObjectProperty }}
            FILTER NOT EXISTS {{ ?otherIndiv rdf:type owl:DatatypeProperty }}
            FILTER NOT EXISTS {{ ?otherIndiv rdf:type owl:AnnotationProperty }}
        }}
    }}

    """

    g.update(query)

In [14]:
def clear_serialize(g, outputDes):
    open(f"{outputDes}", 'w').close() # Clear destination file content
    g.serialize(destination=f"{outputDes}", format="turtle")